## Features

$Jitter(\%) = \frac{(1/(N-1)) \sum_{i=1}^{N-1} |T_i - T_{i+1}|}{(1/N) \sum_{i=1}^{N} T_i}$

$Shimmer(dB) = \frac{1}{N-1} \sum_{i=1}^{N-1} \left| 20 \log\left(\frac{A_{i+1}}{A_i}\right) \right|$

A primeira pergunta que me fiz é se essa biblioteca <code>praat-parselmouth</code> usa essas equações.

A biblioteca não reimplementa esses algoritmos. Ela é um <i>wrapper</i> que chama diretamente o código-fonte original em C++ do Praat, o software de análise fonética (https://www.fon.hum.uva.nl/praat/). Quando você executa parselmouth.praat.call(..., "Get jitter (local)"), você está, de fato, executando o comando "Jitter (local)" do software Praat. Pode ser considerado uma boa referência?

No manual oficial do software, a métrica "Jitter (local, %)" é definida exatamente como a média da diferença absoluta entre períodos consecutivos, dividida pelo período médio.

Da mesma forma, "Shimmer (local, dB)" é definido como a média do valor absoluto da diferença em decibéis (20 * log10) entre as amplitudes de ciclos consecutivos.

<b>Frequência Fundamental (F0)</b>

O texto do artigo menciona o algoritmo [YIN](http://audition.ens.fr/adc/pdf/2002_JASA_YIN.pdf). O Praat usa este algoritmo por padrão no método to_pitch().

O texto também menciona "gênero e idade". Isso é relevante para definir os limites de pitch (pitch_floor e pitch_ceiling).

Valores comuns informados na documentação até o momento:
- Homem: pitch_floor=75, pitch_ceiling=300
- Mulher: pitch_floor=100, pitch_ceiling=500

Usaremos valores genéricos (75-600) se não soubermos o gênero.
        
<b>Harmonic to Noise Ratio (HNR)</b>

O texto menciona o algoritmo de Krom. Também método padrão de HNR no Praat.
        
<b>Jitter e Shimmer</b>
        
No Praat, um PointProcess representa os momentos de pulsos glóticos, ou seja, os instantes exatos em que as pregas vocais se fecham e geram uma vibração periódica.

O que podemos chamar de “lista temporal dos picos de vozeamento" detectados no sinal.

Esses pulsos são o que o Praat usa para medir a estabilidade temporal e de amplitude das vibrações, o que dá origem às medidas de jitter e shimmer.
        
Jitter e Shimmer são calculados a partir das localizações dos pulsos glotais (pitch periods).     
        
Jitter corresponde ao "Jitter (local)" no Praat, que é a variação ciclo-a-ciclo, expresso em porcentagem.
Os parâmetros (0, 0, 0.0001, 0.02, 1.3) são padrões do Praat.
        
Shimmer corresponde ao "Shimmer (local, dB)" no Praat.
Os parâmetros (0, 0, 0.0001, 0.02, 1.3, 1.6) são padrões do Praat.

In [1]:
# pip install praat-parselmouth numpy
import os
import io
import sys
import zipfile
import tempfile
import subprocess

import librosa
import parselmouth

import scipy.signal
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Tentativa 1

In [2]:
# PRAAT
def extract_features_praat(audio_path):
    y, sr = librosa.load(audio_path, sr=None)
    
    f0 = librosa.yin(y, fmin=50, fmax=800, sr=sr)
    f0_voiced = f0[~np.isnan(f0)]
    
    f0_voiced_filtered = f0_voiced[(f0_voiced > 50) & (f0_voiced < 500)]
    
    if len(f0_voiced_filtered) == 0:
        f0_mean = f0_max = f0_min = f0_std = np.nan
    else:
        f0_mean = np.mean(f0_voiced_filtered)
        f0_max  = np.max(f0_voiced_filtered)
        f0_min  = np.min(f0_voiced_filtered)
        f0_std  = np.std(f0_voiced_filtered)

    snd = parselmouth.Sound(audio_path)
    try:
        pointProcess = parselmouth.praat.call(snd, "To PointProcess (periodic, cc)", 75, 500)
        jitter_local = parselmouth.praat.call(pointProcess, "Get jitter (local)", 0, 0, 1.3)
        shimmer_local = parselmouth.praat.call([snd, pointProcess], "Get shimmer (local)", 0, 0, 1.3, 1.6)
        hnr = parselmouth.praat.call(snd, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
        hnr_value = parselmouth.praat.call(hnr, "Get mean", 0, 0)
        nhr = 10 ** (-hnr_value / 10) if hnr_value > 0 else 0
    except:
        jitter_local = shimmer_local = hnr_value = nhr = np.nan

    features = [
        f0_mean,
        f0_max,
        f0_min,
        f0_std,
        jitter_local,
        shimmer_local,
        hnr_value,
        nhr
    ]
    return np.array(features, dtype=np.float64)

# LIBROSA
def extract_features_librosa(audio_path):
    y, sr = librosa.load(audio_path, sr=None)
    
    f0 = librosa.yin(y, fmin=50, fmax=500, sr=sr)
    f0_voiced = f0[~np.isnan(f0)]
    
    if len(f0_voiced) == 0:
        f0_mean = f0_max = f0_min = f0_std = np.nan
    else:
        f0_voiced_filtered = f0_voiced[(f0_voiced > 50) & (f0_voiced < 500)]
        f0_mean = np.mean(f0_voiced_filtered)
        f0_max  = np.max(f0_voiced_filtered)
        f0_min  = np.min(f0_voiced_filtered)
        f0_std  = np.std(f0_voiced_filtered)
    
    if len(f0_voiced_filtered) > 1:
        periods = 1 / f0_voiced_filtered
        jitter_local = np.mean(np.abs(np.diff(periods))) / np.mean(periods) * 100
    else:
        jitter_local = np.nan
    
    shimmer_vals = []
    for i, f in enumerate(f0_voiced_filtered[:-1]):
        start = int(i * sr / f)
        end   = int((i+1) * sr / f)
        if end > len(y):
            break
        cycle_amp = np.max(y[start:end]) - np.min(y[start:end])
        shimmer_vals.append(cycle_amp)
    shimmer_vals = np.array(shimmer_vals)
    if len(shimmer_vals) > 1:
        shimmer_local = np.mean(np.abs(np.diff(shimmer_vals))) / np.mean(shimmer_vals) * 100
    else:
        shimmer_local = np.nan
    
    frame_len = int(0.05 * sr)
    hop_len   = int(0.025 * sr)
    autocorr_energy = []
    total_energy = []
    for i in range(0, len(y)-frame_len, hop_len):
        frame = y[i:i+frame_len]
        total_energy.append(np.sum(frame**2))
        autocorr = np.correlate(frame, frame, mode='full')
        mid = len(autocorr)//2
        harmonic_energy = np.max(autocorr[mid:])
        autocorr_energy.append(harmonic_energy)
    total_energy = np.sum(total_energy)
    harmonic_energy = np.sum(autocorr_energy)
    if total_energy > 0:
        hnr_db = 10 * np.log10(harmonic_energy / (total_energy - harmonic_energy + 1e-8))
        nhr = 10 ** (-hnr_db / 10)
    else:
        hnr_db = np.nan
        nhr = np.nan

    return np.array([
        f0_mean, f0_max, f0_min, f0_std,
        jitter_local, shimmer_local, hnr_db, nhr
    ], dtype=np.float64)

### Teste 1:

In [37]:
file_path = r'C:\Users\joaov_zm1q2wh\python\icassp_challenge\joao\data\ID023_phonationA.wav'

features_names = [
    "Fao(Hz)",
    "Fhi(Hz)",
    "Flo(Hz)",
    "sigma Fo(Hz)",
    "Jitter(%)",
    "Shimmer(%)",
    "HNR(dB)",
    "NHR"
]

try:
    features_librosa = extract_features_librosa(file_path)
    features_praat   = extract_features_praat(file_path)

    print(f"{'Feature':<15} {'Librosa':>12} {'Praat':>12}")
    print("-" * 40)

    for name, val_lib, val_praat in zip(features_names, features_librosa, features_praat):
        str_lib = f"{val_lib:.5f}" if not np.isnan(val_lib) else "N/A"
        str_praat = f"{val_praat:.5f}" if not np.isnan(val_praat) else "N/A"
        print(f"{name:<15} {str_lib:>12} {str_praat:>12}")

except Exception as e:
    print(f"Erro: {e}")

Feature              Librosa        Praat
----------------------------------------
Fao(Hz)            181.41720    181.41720
Fhi(Hz)            243.76916    243.76916
Flo(Hz)             63.44466     63.44466
sigma Fo(Hz)        33.77668     33.77668
Jitter(%)            4.09915          N/A
Shimmer(%)          54.73909          N/A
HNR(dB)            113.74284          N/A
NHR                  0.00000          N/A


Peguei alguns valores próximos na base do problema de Parkinson que achei útil para servir como base:

| Type           | Value
| -------------- | ---------
| Fo(Hz)         | 119.992
| Fhi(Hz)        | 157.302
| Flo(Hz)        | 74.997
| Jitter(%)      | 0.00784
| Jitter:DDP     | 0.01109
| Shimmer(%)     | 0.04374
| Shimmer(dB)    | 0.426
| NHR            | 0.02211
| HNR            | 21.033

Já deu para perceber que metade nem rodou e a outra metade está só a desgraça.

"Ah João, tenha paciência." Paciência o que rapaz, te orienta cabra safado, isso aqui é só a peste, castigo divino.

Depois de tanto ler a API do Praat e a documentação da librosa, percebi que o problema com os cálculos originais residia no seguinte: 
    
| Feature | Problema Original | Solução Aplicada (Praat) |
| :--- | :--- | :--- |
| **F0 (Praat)** | Acessar valores de Pitch incorretamente. | Uso do método `pitch.selected_array['frequency']` após `snd.to_pitch()` para extrair os valores de F0. |
| **Jitter (Praat)** | Chamada incompleta da função `parselmouth.praat.call` para `Get jitter (local)`. | Uso da chamada completa com os 5 argumentos esperados pelo Praat: `0, 0, 0.0001, 0.02, 1.3`.|
| **Shimmer (Praat)** | Chamada incompleta da função `parselmouth.praat.call` para `Get shimmer (local)`. | Uso da chamada completa com os 6 argumentos esperados pelo Praat: `0, 0, 0.0001, 0.02, 1.3, 1.6`.|
| **HNR (Praat)** | Chamada incorreta da função `to_harmonicity()`. | Uso da chamada `parselmouth.praat.call(snd, "To Harmonicity (cc)", 0.01, 75.0, 0.1, 1.0)` com os 4 argumentos esperados.|
| **F0 (Librosa)** | Inconsistência nos limites de `fmax` (800 vs 500 Hz). | Ajuste para `fmax=600` para consistência com o Praat e manutenção da filtragem original (50-500 Hz).|
| **Jitter/Shimmer/HNR (Librosa)** | Implementações manuais de Shimmer e HNR são aproximações grosseiras e incorretas para as definições de Praat. | Mantidas as implementações originais para fins de comparação, mas com a ressalva de que **não fornecem a exatidão** desejada. O Praat é o padrão de referência para estas features.

### Tentativa 2

In [3]:
### PRAAT
def extract_features_praat_corrected(audio_path):
    snd = parselmouth.Sound(audio_path)
    
    try:
        pitch = snd.to_pitch(pitch_floor=75.0, pitch_ceiling=600.0)
    except Exception as e:
        print(f"Erro ao calcular Pitch (to_pitch): {e}", file=sys.stderr)
        return np.full(8, np.nan, dtype=np.float64)

    f0_values = np.array(pitch.selected_array['frequency'], dtype=np.float64)
    f0_values = f0_values[f0_values > 0]
    f0_values_filtered = f0_values[(f0_values > 50) & (f0_values < 500)]
    
    if len(f0_values_filtered) == 0:
        f0_mean = f0_max = f0_min = f0_std = np.nan
    else:
        f0_mean = np.mean(f0_values_filtered)
        f0_max  = np.max(f0_values_filtered)
        f0_min  = np.min(f0_values_filtered)
        f0_std  = np.std(f0_values_filtered)

    try:
        pointProcess = parselmouth.praat.call(snd, "To PointProcess (periodic, cc)", 75.0, 600.0)
        jitter_local = parselmouth.praat.call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)

        shimmer_local_percent = parselmouth.praat.call([snd, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
        
        hnr = parselmouth.praat.call(snd, "To Harmonicity (cc)", 0.01, 75.0, 0.1, 1.0)
        hnr_value = parselmouth.praat.call(hnr, "Get mean", 0, 0)
        
        nhr = 10 ** (-hnr_value / 10) if hnr_value > 0 else np.nan

    except Exception as e:
        print(f"Erro na extração Praat (Jitter/Shimmer/HNR): {e}", file=sys.stderr)
        jitter_local = shimmer_local_percent = hnr_value = nhr = np.nan
        
    features = [
        f0_mean,
        f0_max,
        f0_min,
        f0_std,
        jitter_local,
        shimmer_local_percent,
        hnr_value,
        nhr
    ]
    return np.array(features, dtype=np.float64)

# LIBROSA
def extract_features_librosa_corrected(audio_path):
    y, sr = librosa.load(audio_path, sr=None)
    
    f0 = librosa.yin(y, fmin=75, fmax=600, sr=sr)
    f0_voiced = f0[~np.isnan(f0)]
    f0_voiced_filtered = f0_voiced[(f0_voiced > 50) & (f0_voiced < 500)]
    
    if len(f0_voiced_filtered) == 0:
        f0_mean = f0_max = f0_min = f0_std = np.nan
    else:
        f0_mean = np.mean(f0_voiced_filtered)
        f0_max  = np.max(f0_voiced_filtered)
        f0_min  = np.min(f0_voiced_filtered)
        f0_std  = np.std(f0_voiced_filtered)
    
    if len(f0_voiced_filtered) > 1:
        periods = 1 / f0_voiced_filtered
        jitter_local = np.mean(np.abs(np.diff(periods))) / np.mean(periods) * 100
    else:
        jitter_local = np.nan
    
    shimmer_local = np.nan
    try:
        if len(f0_voiced_filtered) > 1:
            shimmer_vals = []
            for i, f in enumerate(f0_voiced_filtered[:-1]):
                start = int(i * sr / f)
                end   = int((i+1) * sr / f)
                if end > len(y):
                    break
                cycle_amp = np.max(y[start:end]) - np.min(y[start:end])
                shimmer_vals.append(cycle_amp)
            
            shimmer_vals = np.array(shimmer_vals)
            if len(shimmer_vals) > 1:
                shimmer_local = np.mean(np.abs(np.diff(shimmer_vals))) / np.mean(shimmer_vals) * 100
            else:
                shimmer_local = np.nan
        else:
            shimmer_local = np.nan
    except Exception as e:
        shimmer_local = np.nan
    
    hnr_db = np.nan
    nhr = np.nan
    try:
        frame_len = int(0.05 * sr)
        hop_len   = int(0.025 * sr)
        autocorr_energy = []
        total_energy = []
        for i in range(0, len(y)-frame_len, hop_len):
            frame = y[i:i+frame_len]
            total_energy.append(np.sum(frame**2))
            autocorr = np.correlate(frame, frame, mode='full')
            mid = len(autocorr)//2
            harmonic_energy = np.max(autocorr[mid:])
            autocorr_energy.append(harmonic_energy)
            
        total_energy_sum = np.sum(total_energy)
        harmonic_energy_sum = np.sum(autocorr_energy)
        
        if total_energy_sum > 0 and harmonic_energy_sum < total_energy_sum:
            hnr_db = 10 * np.log10(harmonic_energy_sum / (total_energy_sum - harmonic_energy_sum + 1e-8))
            nhr = 10 ** (-hnr_db / 10)
        else:
            hnr_db = np.nan
            nhr = np.nan
            
    except Exception as e:
        hnr_db = np.nan
        nhr = np.nan

    features = [
        f0_mean, f0_max, f0_min, f0_std,
        jitter_local, shimmer_local, hnr_db, nhr
    ]
    return np.array(features, dtype=np.float64)

### TESTE 2

In [49]:
audio_path = r'C:\Users\joaov_zm1q2wh\python\icassp_challenge\joao\data\ID023_phonationA.wav'

features_names = [
    "Fao(Hz)",
    "Fhi(Hz)",
    "Flo(Hz)",
    "sigma Fo(Hz)",
    "Jitter(%)",
    "Shimmer(%)",
    "HNR(dB)",
    "NHR"
]

try:
    features_librosa_corr = extract_features_librosa_corrected(audio_path)
    features_praat_corr   = extract_features_praat_corrected(audio_path)

    print(f"{'Feature':<15} {'Librosa':>18} {'Praat':>15}")
    print("-" * 50)

    for name, val_lib, val_praat in zip(features_names, features_librosa_corr, features_praat_corr):
        str_lib = f"{val_lib:.5f}" if not np.isnan(val_lib) else "N/A"
        str_praat = f"{val_praat:.5f}" if not np.isnan(val_praat) else "N/A"
        print(f"{name:<15} {str_lib:>18} {str_praat:>15}")

except Exception as e:
    print(f"Erro na execução do teste corrigido: {e}", file=sys.stderr)

Feature                    Librosa           Praat
--------------------------------------------------
Fao(Hz)                  182.14816       192.03097
Fhi(Hz)                  243.76916       217.10861
Flo(Hz)                   79.62720       149.49022
sigma Fo(Hz)              32.59501         4.79336
Jitter(%)                  2.09470         0.00368
Shimmer(%)                28.54700         0.02479
HNR(dB)                        N/A        22.78959
NHR                            N/A         0.00526


A comparação demonstra que a função corrigida do Praat fornece valores que PARECEM ser coerentes para todas as features, enquanto a implementação do Librosa falha em calcular `HNR` e fornece valores de `Jitter` e `Shimmer` significativamente diferentes, confirmando a necessidade de avaliar melhor o dados que esperamos de fato. Segue novamente os valores usados como base:

| Type           | Value
| -------------- | ---------
| Fo(Hz)         | 119.992
| Fhi(Hz)        | 157.302
| Flo(Hz)        | 74.997
| Jitter(%)      | 0.00784
| Jitter:DDP     | 0.01109
| Shimmer(%)     | 0.04374
| Shimmer(dB)    | 0.426
| NHR            | 0.02211
| HNR            | 21.033

### Extração

Baseado na avaliação anterior, vou seguir com a função do Praat e ver o que conseguimos...

In [11]:
zip_path = r"C:\Users\joaov_zm1q2wh\python\icassp_challenge\joao\data\SAND_Challenge_task1_dataset.zip"
target_folder = "task1/training/rhythmTA"
metadata_path = "task1/sand_task_1.xlsx"

features_names = [
    "F0_mean_Hz", 
    "F0_max_Hz", 
    "F0_min_Hz", 
    "F0_std_Hz",
    "Jitter_percent", 
    "Shimmer_percent", 
    "HNR_dB", 
    "NHR"
]

data = []

with zipfile.ZipFile(zip_path, 'r') as zipf:
    with zipf.open(metadata_path) as meta_file:
        metadata_df = pd.read_excel(meta_file)
    
    for file in zipf.namelist():
        if file.startswith(target_folder) and file.endswith(".wav"):
            try:
                with zipf.open(file) as audio_file:
                    with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as tmp:
                        tmp.write(audio_file.read())
                        tmp_path = tmp.name

                features = extract_features_praat_corrected(tmp_path)

                os.remove(tmp_path)

                filename = os.path.basename(file)
                sample_id = filename.split("_")[0]

                meta_row = metadata_df.loc[metadata_df["ID"] == sample_id]
                if not meta_row.empty:
                    age  = int(meta_row["Age"].values[0])
                    sex  = meta_row["Sex"].values[0]
                    clas = int(meta_row["Class"].values[0])
                else:
                    age = sex = clas = np.nan

                row = [sample_id, age, sex, clas] + list(features)
                data.append(row)

            except Exception as e:
                print(f"Erro no arquivo {file}: {e}", file=sys.stderr)


columns = ["ID", "Age", "Sex", "Class"] + features_names
df = pd.DataFrame(data, columns=columns)

output_csv = r"C:\Users\joaov_zm1q2wh\python\icassp_challenge\joao\data\features_praat_dataset.csv"
df.to_csv(output_csv, index=False, encoding="utf-8-sig")

# O arquivos precisa ter 272 linhas, sem contar com o cabeçalho é claro
print(f"\n✅ Extração concluída! CSV salvo em:\n{output_csv}")
print(df.head())


✅ Extração concluída! CSV salvo em:
C:\Users\joaov_zm1q2wh\python\icassp_challenge\joao\data\features_praat_dataset.csv
      ID  Age Sex  Class  F0_mean_Hz   F0_max_Hz  F0_min_Hz  F0_std_Hz  \
0  ID302   76   F      5  203.650991  257.013782  91.565732  11.491251   
1  ID275   41   F      5  158.636831  223.278832  76.377658  24.308702   
2  ID227   81   F      3  174.898673  317.496039  74.809947  63.765307   
3  ID092   65   M      3  113.833209  333.168834  81.697761  17.878683   
4  ID038   69   F      4   91.013141  127.638661  74.771950  13.778429   

   Jitter_percent  Shimmer_percent     HNR_dB       NHR  
0        0.008911         0.062493  15.608192  0.027490  
1        0.008113         0.086885  12.516582  0.056020  
2        0.023304         0.093255  10.397849  0.091246  
3        0.024890         0.098045   9.616296  0.109237  
4        0.075140         0.220215   0.203258  0.954277  
